In [1]:
# ============ 설정 ============
FOLDER = "/Users/snu.sim/git/ai-chatbot-tasks/task3/images"          # 이미지가 들어있는 폴더 경로
INTERACTIVE = True           # 노트북에서 감정 선택을 직접 입력
USE_OPENAI = True           # OpenAI 캡셔닝 사용 여부 (기본 False)
SAVE_TXT = False              # 결과를 txt로 저장
SAVE_JSON = False            # 결과를 json으로 저장
RANDOM_SEED = 42             # 재현성

In [3]:
# ============ 준비 ============
import os, glob, re, json, random, textwrap
from dataclasses import dataclass
from typing import List, Dict, Optional
from dotenv import load_dotenv

random.seed(RANDOM_SEED)

# .env 파일 로드
load_dotenv()

# .env 파일에서 API 키 가져오기
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# OpenAI 클라이언트 초기화
client = None
if USE_OPENAI and OPENAI_API_KEY:
    try:
        from openai import OpenAI
        client = OpenAI(api_key=OPENAI_API_KEY)
        print(f"[성공] OpenAI 클라이언트 초기화 완료")
    except Exception as e:
        print(f"[경고] OpenAI 클라이언트를 불러오지 못했습니다: {e}")
        client = None
elif USE_OPENAI:
    # 키가 없거나 플레이스홀더 텍스트가 그대로 남아있는 경우
    print("[알림] OPENAI_API_KEY가 .env 파일에 설정되지 않았습니다. .env 파일을 확인해주세요.")

[성공] OpenAI 클라이언트 초기화 완료


In [4]:
# ============ 데이터 클래스 정의 ============
@dataclass
class StoryTheme:
    """사용자에게 제안할 이야기 테마를 저장하는 데이터 클래스"""
    emotion: str
    summary: str

In [5]:
# ============ 1. 이미지 분석 ============
import base64

class ImageAnalyzer:
    """GPT Vision API를 사용하여 이미지 캡션을 생성하는 클래스"""
    def __init__(self, client: OpenAI):
        self.client = client
        self.prompt_style = "이 이미지를 보고, 한국어로 1~2문장으로 자연스럽게 묘사해줘."

    def _encode_image(self, path: str) -> str:
        with open(path, "rb") as f:
            return base64.b64encode(f.read()).decode("utf-8")

    def analyze_image(self, path: str) -> str:
        b64_image = self._encode_image(path)
        try:
            response = self.client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "user", "content": [
                        {"type": "text", "text": self.prompt_style},
                        {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{b64_image}"}}
                    ]}
                ],
                max_tokens=150
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            return f"(이미지 분석 실패: {os.path.basename(path)} - {e})"

    def analyze_folder(self, folder: str, exts: List[str] = [".png", ".jpg", ".jpeg"]) -> List[str]:
        if not os.path.isdir(folder):
            raise FileNotFoundError(f"폴더를 찾을 수 없습니다: {folder}")
        
        files = sorted([f for f in os.listdir(folder) if any(f.lower().endswith(ext) for ext in exts)])
        paths = [os.path.join(folder, f) for f in files]
        
        captions = [self.analyze_image(p) for p in paths]
        return captions

In [6]:
# ============ 2. 스토리 기획 및 생성 ============
class StoryGenerator:
    """캡션을 바탕으로 이야기 테마를 제안하고, 최종 스토리를 생성하는 클래스"""
    def __init__(self, client: OpenAI, model: str = "gpt-4o-mini"):
        self.client = client
        self.model = model

    def propose_themes(self, captions: List[str], k: int = 3) -> List[StoryTheme]:
        """캡션을 바탕으로 이야기 테마(감정+요약)를 k개 제안받고 파싱하여 반환"""
        captions_str = "\n".join(f"- {c}" for c in captions)
        
        prompt = f"""
        아래는 여러 이미지에 대한 묘사입니다.
        이 묘사들을 바탕으로 만들 수 있는 흥미로운 이야기의 '감정'과 '한 줄 요약'을 {k}가지만 제안해주세요.

        [이미지 묘사]
        {captions_str}

        [출력 형식]
        반드시 아래와 같이 번호, "감정:", "요약:" 키워드를 포함한 형식으로만 답변해주세요.

        1. 감정: [첫 번째 제안 감정]
        2. 감정: [두 번째 제안 감정]
        ...
        """
        
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=500,
            temperature=0.7,
        )
        content = response.choices[0].message.content
        
        # 정규표현식을 사용하여 AI의 텍스트 응답을 파싱
        themes = []
        pattern = re.compile(r"\d+\.\s*감정:\s*(.*?)\n\s*요약:\s*(.*)", re.MULTILINE)
        matches = pattern.findall(content)
        
        for match in matches:
            themes.append(StoryTheme(emotion=match[0].strip(), summary=match[1].strip()))
            
        return themes

    def generate_story(self, captions: List[str], theme: StoryTheme) -> str:
        """선택된 테마와 캡션을 바탕으로 최종 이야기를 생성"""
        captions_str = "\n".join(f"{i+1}. {c}" for i, c in enumerate(captions))
        
        prompt = f"""
        당신은 뛰어난 스토리텔러입니다. 지금부터 제가 드리는 정보들을 조합해서 하나의 완성된 이야기를 만들어야 합니다.

        [과업 지시]
        1. 이야기의 전체적인 주제는 아래 [선택된 테마]를 반드시 따라야 합니다.
        2. 이야기의 각 장면은 아래 [이미지 묘사]의 순서를 충실히 반영해야 합니다.
        3. 문체는 '3인칭 관찰자 시점'과 '현재 시제'를 사용해주세요.
        4. 각 이미지 묘사에 해당하는 이야기는 1~2 문장으로 간결하게 작성해주세요.

        [선택된 테마]
        - 감정: {theme.emotion}
        - 요약: {theme.summary}

        [이미지 묘사]
        {captions_str}

        [출력]
        최종 이야기만 출력해주세요.
        """
        
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "당신은 주어진 조건에 따라 감동적인 이야기를 만드는 스토리텔러입니다."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=1000,
            temperature=0.8,
        )
        return response.choices[0].message.content.strip()

In [7]:
# ============ 3. 실행부 ============

def jaccard_similarity(text1, text2):
    """
    두 텍스트 사이의 자카드 유사도를 계산하는 함수.
    단어 집합을 기반으로 얼마나 많은 단어를 공유하는지 측정합니다.
    """
    # 공백을 기준으로 단어 집합을 생성
    set1 = set(text1.split())
    set2 = set(text2.split())
    
    if not set1 or not set2:
        return 0.0
    
    # 교집합과 합집합을 계산
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    
    return intersection / union if union != 0 else 0.0


if __name__ == "__main__":
    if not client:
        raise RuntimeError("OpenAI 클라이언트가 초기화되지 않았습니다. API 키를 확인해주세요.")

    # 1. 분석: 모든 이미지에 대한 캡션 생성
    print(">>> 1. 이미지 분석을 시작합니다...")
    analyzer = ImageAnalyzer(client)
    captions = analyzer.analyze_folder(FOLDER)
    print("분석 완료!\n")
    print("--- 이미지 캡션 ---")
    for i, cap in enumerate(captions, 1):
        print(f"{i}) {cap}")
    print("-" * 20)

    sg = StoryGenerator(client)

    # 2. 제안: 캡션을 바탕으로 이야기 테마 제안
    print("\n>>> 2. AI가 이미지에서 발견한 영감을 스케치합니다...")
    themes = sg.propose_themes(captions)
    
    if not themes:
        raise RuntimeError("AI로부터 이야기 테마를 제안받지 못했습니다. API 상태나 입력 내용을 확인해주세요.")
    
    # 3. 선택: 사용자에게 테마 선택 요청 (UX 개선)
    print("제안 완료!\n")
    print("--- 어떤 감동을 원하시나요? ---")
    for i, theme in enumerate(themes, 1):
        print(f"{i}. {theme.emotion}")
        print(f"   ({theme.summary})")
    print("-" * 20)

    chosen_theme = None
    if INTERACTIVE and themes:
        while True:
            # 숫자 대신 자유 텍스트를 입력받도록 프롬프트 변경
            user_input = input("\n>> 어떤 분위기의 이야기로 만들어볼까요? (번호나 느낌을 자유롭게 입력해주세요): ")
            
            if not user_input.strip():
                print("입력이 비어있습니다. 다시 입력해주세요.")
                continue

            # 사용자가 숫자를 입력한 경우, 해당 번호의 테마를 직접 선택
            try:
                choice_idx = int(user_input) - 1
                if 0 <= choice_idx < len(themes):
                    chosen_theme = themes[choice_idx]
                    print(f"\n[선택] '{chosen_theme.emotion}' 테마를 선택하셨습니다.")
                    break
                else:
                    print(f"숫자를 입력하시려면 1에서 {len(themes)} 사이로 입력해주세요.")
                    continue
            except ValueError:
                # 사용자가 텍스트를 입력한 경우, 유사도 계산
                similarities = []
                for theme in themes:
                    # '감정'과 '요약'을 합쳐서 더 정확한 유사도 측정
                    theme_text = f"{theme.emotion} {theme.summary}"
                    score = jaccard_similarity(user_input, theme_text)
                    similarities.append(score)
                
                # 가장 높은 유사도를 가진 테마를 선택
                best_match_index = similarities.index(max(similarities))
                chosen_theme = themes[best_match_index]
                print(f"\n[분석] 입력하신 내용과 가장 비슷한 '{chosen_theme.emotion}' 테마를 선택했습니다.")
                break
    else:
        # 비대화형 모드에서는 첫 번째 제안을 자동으로 선택
        chosen_theme = themes[0]
    
    # 4. 생성: 선택된 테마로 최종 이야기 생성
    print(f"\n>>> 3. '{chosen_theme.emotion}' 테마로 이야기를 생성합니다...")
    story = sg.generate_story(captions, chosen_theme)
    print("생성 완료!\n")

    print("="*10 + " 최종 생성된 이야기 " + "="*10)
    wrapped_story = textwrap.fill(story, width=80)
    print(wrapped_story)
    print("="*38)
    
    # 5. 저장 (설정에 따라)
    # (기존 저장 로직은 변경 없이 유지)
    if SAVE_TXT:
        path = os.path.join(FOLDER, "story_output.txt")
        with open(path, "w", encoding="utf-8") as f:
            f.write(story)
        print(f"\n[성공] 이야기가 {path} 에 저장되었습니다.")

    if SAVE_JSON:
        path = os.path.join(FOLDER, "story_output.json")
        output_data = {
            "captions": captions,
            "chosen_theme": asdict(chosen_theme),
            "story": story,
            "all_proposals": [asdict(t) for t in themes]
        }
        with open(path, "w", encoding="utf-8") as f:
            json.dump(output_data, f, ensure_ascii=False, indent=2)
        print(f"\n[성공] 결과 데이터가 {path} 에 저장되었습니다.")

>>> 1. 이미지 분석을 시작합니다...
분석 완료!

--- 이미지 캡션 ---
1) 인형방에서 다양한 장난감들이 모여 있는 모습이 담긴 이미지입니다. 캐릭터들은 각기 다른 모습과 색깔로 화기애애한 분위기를 연출하고 있습니다.
2) 다양한 장난감들이 선반 위에 나란히 서서 함께하고 있는 모습입니다. 각기 다른 캐릭터들이 친근하게 지켜보며 모험을 준비하는 듯한 분위기를 자아냅니다.
3) 이미지에는 친근한 두 캐릭터가 함께 서 있는 모습이 담겨 있습니다. 한 명은 카우보이 복장을 하고 있고, 다른 한 명은 우주복을 입고 있어 서로 다른 배경을 가진 친구들처럼 보입니다.
--------------------

>>> 2. AI가 이미지에서 발견한 영감을 스케치합니다...
제안 완료!

--- 어떤 감동을 원하시나요? ---
1. 기쁨
   (다양한 장난감들이 함께 모여 서로의 모험을 응원하며 화기애애한 분위기를 만들어간다.)
2. 호기심
   (서로 다른 배경을 가진 캐릭터들이 힘을 합쳐 새로운 모험을 준비하며 궁금증을 자아낸다.)
3. 우정
   (카우보이와 우주복을 입은 두 친구가 서로 다른 세계에서 온 만큼 특별한 우정을 나누며 모험을 함께 계획한다.)
--------------------

[분석] 입력하신 내용과 가장 비슷한 '기쁨' 테마를 선택했습니다.

>>> 3. '기쁨' 테마로 이야기를 생성합니다...
생성 완료!

========== 최종 생성된 이야기 ==========
인형방의 한가운데, 다양한 장난감들이 모여 서로의 모험을 응원하며 화기애애한 분위기를 만들어간다. 각기 다른 모습과 색깔로 장식된 인형들은
서로의 존재를 반기며 기쁨으로 가득 차 있다.  선반 위에는 각양각색의 장난감들이 나란히 서 있다. 그들은 사이좋게 지켜보며, 이제 시작될
모험을 위해 준비하는 모습이다. 서로의 눈빛에서 기대와 소망이 반짝인다.  두 캐릭터가 친근하게 나란히 서 있다. 카우보이 복장을 한 친구는
자신감 넘치는 표정을 짓고, 우주복을 입은 친구는 호기심 가